In [106]:
import sys
!{sys.executable} -m pip install efficient_apriori

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [107]:
import pandas as pd
import numpy as np
import glob as gl
import pickle
from efficient_apriori import apriori
import warnings
import os
import pathlib
warnings.filterwarnings('ignore')

In [108]:
sub_MR1 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSub_MR1.csv', index_col= 0)
sub_MR2 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSub_MR2.csv', index_col= 0)
sub_MR3 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSub_MR3.csv', index_col= 0)
sub_MR4 = pd.read_csv('/Users/alduck/Documents/PhDStuff_repo/PasiveTestingUsingMTplusFuzzTesting/individual_logs/FinalLogSub_MR4.csv', index_col= 0)

In [109]:
sub_MR1

,"(a,b)",input_a,input_b,operand,const,output,MR1_input_a,MR1_input_b,MR1_inputs,MR1_output,MR1_checker
0,"(9,2)",9,2,sub,6,7.0,2.0,9.0,"(2,9)",-7.0,Violated
1,"(8,1)",8,1,sub,6,7.0,1.0,8.0,"(1,8)",-7.0,Violated
2,"(4,8)",4,8,sub,6,-4.0,8.0,4.0,"(8,4)",4.0,Violated
3,"(7,2)",7,2,sub,6,5.0,2.0,7.0,"(2,7)",-5.0,Violated
4,"(0,6)",0,6,sub,6,-6.0,6.0,0.0,"(6,0)",6.0,Violated
...,...,...,...,...,...,...,...,...,...,...,...
95,"(2,7)",2,7,sub,6,-5.0,7.0,2.0,"(7,2)",5.0,Violated
96,"(2,3)",2,3,sub,6,-1.0,3.0,2.0,"(3,2)",1.0,Violated
97,"(1,3)",1,3,sub,6,-2.0,3.0,1.0,"(3,1)",2.0,Violated
98,"(0,6)",0,6,sub,6,-6.0,6.0,0.0,"(6,0)",6.0,Violated


In [110]:
def prepro(data):
    MR_name = list(sub_MR1.keys())[-1]

    for index, row in data.iterrows():
        auxR1 = row[MR_name]
        data.at[index, MR_name] = MR_name + ' == ' + auxR1
        data.at[index, 'input_a'] =  'input_a == ' + str(row['input_a'])
        data.at[index, 'input_b'] = 'input_b == ' + str(row['input_b'])
        data.at[index, 'output'] = 'output == ' + str(row['output'])
        data.at[index, 'MR1_input_a'] =  'MR1_input_a == ' + str(row['MR1_input_a'])
        data.at[index, 'MR1_input_b'] = 'MR1_input_b == ' + str(row['MR1_input_b'])
        data.at[index, 'MR1_inputs'] = 'MR1_inputs == ' + str(row['MR1_inputs'])
        data.at[index, 'MR1_output'] = 'MR1_output == ' + str(row['MR1_output'])

    return data

In [111]:
data = prepro(sub_MR1)
data1 = data[['(a,b)', 'input_a', 'input_b', 'output', 'MR1_input_a', 'MR1_input_b', 'MR1_inputs', 'MR1_output','MR1_checker']]
data2 = data[['input_a', 'input_b', 'output', 'MR1_input_a', 'MR1_input_b', 'MR1_output','MR1_checker']]
data3 = data[['input_a', 'input_b', 'output', 'MR1_input_a', 'MR1_input_b','MR1_checker']]
data4 = data[['input_a', 'input_b', 'MR1_input_a', 'MR1_input_b', 'MR1_checker']]
data5 = data[["(a,b)",'MR1_checker']]
data6 = data[['input_a', 'input_b', 'MR1_checker']]

# data_2 = prepro(data_2)
# data_3 = prepro(data_3)
# data_4 = prepro(data_4)
# data_5 = prepro(data_5)

In [112]:
data

,"(a,b)",input_a,input_b,operand,const,output,MR1_input_a,MR1_input_b,MR1_inputs,MR1_output,MR1_checker
0,"(9,2)",input_a == 9,input_b == 2,sub,6,output == 7.0,MR1_input_a == 2.0,MR1_input_b == 9.0,"MR1_inputs == (2,9)",MR1_output == -7.0,MR1_checker == Violated
1,"(8,1)",input_a == 8,input_b == 1,sub,6,output == 7.0,MR1_input_a == 1.0,MR1_input_b == 8.0,"MR1_inputs == (1,8)",MR1_output == -7.0,MR1_checker == Violated
2,"(4,8)",input_a == 4,input_b == 8,sub,6,output == -4.0,MR1_input_a == 8.0,MR1_input_b == 4.0,"MR1_inputs == (8,4)",MR1_output == 4.0,MR1_checker == Violated
3,"(7,2)",input_a == 7,input_b == 2,sub,6,output == 5.0,MR1_input_a == 2.0,MR1_input_b == 7.0,"MR1_inputs == (2,7)",MR1_output == -5.0,MR1_checker == Violated
4,"(0,6)",input_a == 0,input_b == 6,sub,6,output == -6.0,MR1_input_a == 6.0,MR1_input_b == 0.0,"MR1_inputs == (6,0)",MR1_output == 6.0,MR1_checker == Violated
...,...,...,...,...,...,...,...,...,...,...,...
95,"(2,7)",input_a == 2,input_b == 7,sub,6,output == -5.0,MR1_input_a == 7.0,MR1_input_b == 2.0,"MR1_inputs == (7,2)",MR1_output == 5.0,MR1_checker == Violated
96,"(2,3)",input_a == 2,input_b == 3,sub,6,output == -1.0,MR1_input_a == 3.0,MR1_input_b == 2.0,"MR1_inputs == (3,2)",MR1_output == 1.0,MR1_checker == Violated
97,"(1,3)",input_a == 1,input_b == 3,sub,6,output == -2.0,MR1_input_a == 3.0,MR1_input_b == 1.0,"MR1_inputs == (3,1)",MR1_output == 2.0,MR1_checker == Violated
98,"(0,6)",input_a == 0,input_b == 6,sub,6,output == -6.0,MR1_input_a == 6.0,MR1_input_b == 0.0,"MR1_inputs == (6,0)",MR1_output == 6.0,MR1_checker == Violated


In [113]:
def ruleGenerator(transactions_df, support, conf):
    """
        Generate set of rules with given support and confidence using transactions_df
    """
    # print(transactions_df.head(20))
    transactions_array = transactions_df.to_numpy()
    transactions_list = transactions_array.tolist()
    itemSets, rules = apriori(transactions_list, min_support=support, min_confidence=conf)
    return itemSets, rules


In [114]:
items1, rules1 = ruleGenerator(data1, 0.1, float(1))
items2, rules2 = ruleGenerator(data2, 0.1, float(1))
items3, rules3 = ruleGenerator(data3, 0.1, float(1))
items4, rules4 = ruleGenerator(data4, 0.1, float(1))
items5, rules5 = ruleGenerator(data5, 0.01, float(1))
items6, rules6 = ruleGenerator(data6, 0.01, float(1))

In [115]:
def ruleGen(items, rules1):
    sets1 = []
    sets2 = []
    dfListUnion = []
    dfListIntersection = []
    sets1.append(set(rules1))
    sets2.append(set(items))

    union = set.union(*sets1)
    lUnion = list(union)

    for rul in range(0, len(lUnion)):
                dfAux_union = {'LHS': tuple(lUnion[rul].lhs), 'LHS_size': len(lUnion[rul].lhs),
                            'RHS': tuple(lUnion[rul].rhs), 'RHS_size': len(lUnion[rul].rhs),
                            'conf': lUnion[rul].confidence,
                            'sup': lUnion[rul].support, 'lift': lUnion[rul].lift, 'cov': lUnion[rul].conviction}
                dfListUnion.append(dfAux_union)

    dfRule = pd.DataFrame(dfListUnion)

    return dfRule

In [116]:
dfRule1 = ruleGen(items1, rules1)
dfRule2 = ruleGen(items2, rules2)
dfRule3 = ruleGen(items3, rules3)
dfRule4 = ruleGen(items4, rules4)
dfRule5 = ruleGen(items5, rules5)
dfRule6 = ruleGen(items6, rules6)

In [117]:
dfRule1.to_csv('rules_data1_sub.csv')
dfRule2.to_csv('rules_data2_sub.csv')
dfRule3.to_csv('rules_data3_sub.csv')
dfRule4.to_csv('rules_data4_sub.csv')
dfRule5.to_csv('rules_data5_sub.csv')
dfRule6.to_csv('rules_data6_sub.csv')

In [118]:
dfRule1

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_a == 5,)",1,"(MR1_checker == Violated,)",1,1.0,0.12,1.063830,60000000.0
1,"(input_b == 7,)",1,"(MR1_input_a == 7.0,)",1,1.0,0.14,7.142857,860000000.0
2,"(MR1_input_a == 7.0,)",1,"(input_b == 7,)",1,1.0,0.14,7.142857,860000000.0
3,"(input_a == 1,)",1,"(MR1_input_b == 1.0,)",1,1.0,0.12,8.333333,880000000.0
4,"(MR1_input_b == 1.0,)",1,"(input_a == 1,)",1,1.0,0.12,8.333333,880000000.0
...,...,...,...,...,...,...,...,...
66,"(MR1_checker == Violated, MR1_input_a == 7.0)",2,"(input_b == 7,)",1,1.0,0.14,7.142857,860000000.0
67,"(input_b == 7,)",1,"(MR1_checker == Violated, MR1_input_a == 7.0)",2,1.0,0.14,7.142857,860000000.0
68,"(MR1_input_a == 7.0,)",1,"(MR1_checker == Violated, input_b == 7)",2,1.0,0.14,7.142857,860000000.0
69,"(input_b == 6,)",1,"(MR1_input_a == 6.0,)",1,1.0,0.12,8.333333,880000000.0


In [119]:
dfRule2

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_a == 5,)",1,"(MR1_checker == Violated,)",1,1.0,0.12,1.063830,60000000.0
1,"(input_b == 7,)",1,"(MR1_input_a == 7.0,)",1,1.0,0.14,7.142857,860000000.0
2,"(MR1_input_a == 7.0,)",1,"(input_b == 7,)",1,1.0,0.14,7.142857,860000000.0
3,"(input_a == 1,)",1,"(MR1_input_b == 1.0,)",1,1.0,0.12,8.333333,880000000.0
4,"(MR1_input_b == 1.0,)",1,"(input_a == 1,)",1,1.0,0.12,8.333333,880000000.0
...,...,...,...,...,...,...,...,...
66,"(MR1_checker == Violated, MR1_input_a == 7.0)",2,"(input_b == 7,)",1,1.0,0.14,7.142857,860000000.0
67,"(input_b == 7,)",1,"(MR1_checker == Violated, MR1_input_a == 7.0)",2,1.0,0.14,7.142857,860000000.0
68,"(MR1_input_a == 7.0,)",1,"(MR1_checker == Violated, input_b == 7)",2,1.0,0.14,7.142857,860000000.0
69,"(input_b == 6,)",1,"(MR1_input_a == 6.0,)",1,1.0,0.12,8.333333,880000000.0


In [120]:
dfRule3

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_b == 2,)",1,"(MR1_input_a == 2.0,)",1,1.0,0.13,7.692308,870000000.0
1,"(MR1_input_a == 2.0,)",1,"(input_b == 2,)",1,1.0,0.13,7.692308,870000000.0
2,"(MR1_checker == Violated, input_a == 2)",2,"(MR1_input_b == 2.0,)",1,1.0,0.13,7.142857,860000000.0
3,"(MR1_checker == Violated, MR1_input_b == 2.0)",2,"(input_a == 2,)",1,1.0,0.13,7.142857,860000000.0
4,"(input_a == 1,)",1,"(MR1_checker == Violated,)",1,1.0,0.12,1.063830,60000000.0
5,"(MR1_checker == Violated, input_b == 4)",2,"(MR1_input_a == 4.0,)",1,1.0,0.10,9.090909,890000000.0
6,"(input_a == 9,)",1,"(MR1_input_b == 9.0,)",1,1.0,0.10,10.000000,900000000.0
7,"(MR1_input_b == 9.0,)",1,"(input_a == 9,)",1,1.0,0.10,10.000000,900000000.0
8,"(MR1_input_b == 1.0,)",1,"(MR1_checker == Violated,)",1,1.0,0.12,1.063830,60000000.0
9,"(input_a == 5,)",1,"(MR1_checker == Violated,)",1,1.0,0.12,1.063830,60000000.0


In [121]:
dfRule4

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_b == 2,)",1,"(MR1_input_a == 2.0,)",1,1.0,0.13,7.692308,870000000.0
1,"(MR1_input_a == 2.0,)",1,"(input_b == 2,)",1,1.0,0.13,7.692308,870000000.0
2,"(MR1_checker == Violated, input_a == 2)",2,"(MR1_input_b == 2.0,)",1,1.0,0.13,7.142857,860000000.0
3,"(MR1_checker == Violated, MR1_input_b == 2.0)",2,"(input_a == 2,)",1,1.0,0.13,7.142857,860000000.0
4,"(input_a == 1,)",1,"(MR1_checker == Violated,)",1,1.0,0.12,1.063830,60000000.0
5,"(MR1_checker == Violated, input_b == 4)",2,"(MR1_input_a == 4.0,)",1,1.0,0.10,9.090909,890000000.0
6,"(input_a == 9,)",1,"(MR1_input_b == 9.0,)",1,1.0,0.10,10.000000,900000000.0
7,"(MR1_input_b == 9.0,)",1,"(input_a == 9,)",1,1.0,0.10,10.000000,900000000.0
8,"(MR1_input_b == 1.0,)",1,"(MR1_checker == Violated,)",1,1.0,0.12,1.063830,60000000.0
9,"(input_a == 5,)",1,"(MR1_checker == Violated,)",1,1.0,0.12,1.063830,60000000.0


In [122]:
dfRule5

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"((9,8),)",1,"(MR1_checker == Violated,)",1,1.0,0.01,1.06383,60000000.0
1,"((9,3),)",1,"(MR1_checker == Violated,)",1,1.0,0.01,1.06383,60000000.0
2,"((0,6),)",1,"(MR1_checker == Violated,)",1,1.0,0.03,1.06383,60000000.0
3,"((2,5),)",1,"(MR1_checker == Violated,)",1,1.0,0.03,1.06383,60000000.0
4,"((7,4),)",1,"(MR1_checker == Violated,)",1,1.0,0.01,1.06383,60000000.0
...,...,...,...,...,...,...,...,...
58,"((4,5),)",1,"(MR1_checker == Violated,)",1,1.0,0.01,1.06383,60000000.0
59,"((5,8),)",1,"(MR1_checker == Violated,)",1,1.0,0.02,1.06383,60000000.0
60,"((9,4),)",1,"(MR1_checker == Violated,)",1,1.0,0.02,1.06383,60000000.0
61,"((0,1),)",1,"(MR1_checker == Violated,)",1,1.0,0.02,1.06383,60000000.0


In [123]:
dfRule6

,LHS,LHS_size,RHS,RHS_size,conf,sup,lift,cov
0,"(input_a == 9, input_b == 2)",2,"(MR1_checker == Violated,)",1,1.0,0.01,1.063830,60000000.0
1,"(input_a == 0, input_b == 7)",2,"(MR1_checker == Violated,)",1,1.0,0.01,1.063830,60000000.0
2,"(input_a == 4, input_b == 2)",2,"(MR1_checker == Violated,)",1,1.0,0.01,1.063830,60000000.0
3,"(input_a == 0, input_b == 1)",2,"(MR1_checker == Violated,)",1,1.0,0.02,1.063830,60000000.0
4,"(input_a == 9, input_b == 9)",2,"(MR1_checker == No-violated,)",1,1.0,0.02,16.666667,940000000.0
...,...,...,...,...,...,...,...,...
78,"(MR1_checker == No-violated, input_b == 6)",2,"(input_a == 6,)",1,1.0,0.01,9.090909,890000000.0
79,"(MR1_checker == No-violated, input_a == 6)",2,"(input_b == 6,)",1,1.0,0.01,8.333333,880000000.0
80,"(input_b == 3,)",1,"(MR1_checker == Violated,)",1,1.0,0.08,1.063830,60000000.0
81,"(input_a == 1, input_b == 3)",2,"(MR1_checker == Violated,)",1,1.0,0.01,1.063830,60000000.0
